<a href="https://colab.research.google.com/github/chinmay-sh/StockAnalysisAI/blob/main/TickerAnalysisAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade gradio -q
!pip install yfinance -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.9 MB/s eta 0:00:00


In [55]:
from google.colab import userdata
import yfinance as yf
import gradio as gr
import google.generativeai as genai
from google.colab import userdata
import requests
import math
from pydantic import BaseModel
from typing import List

In [70]:
def predict_stock_price(ticker):
    # Get historical data
    stock_data = yf.Ticker(ticker)
    hist = stock_data.history(period="5d")

    hist_closing_prices = {}
    for key, value in hist["Close"].to_dict().items():
        hist_closing_prices[str(key).split(" ")[0]] = math.floor(value)

    # Get current price
    current_price = stock_data.info['currentPrice']

    # Get news sentiment from Alpha Vantage (replace with actual API call)
    # ...

    # Prepare prompt for LLM
    api_key_name = 'GEMINI_API_KEY'
    system_instructions = """
    You are a stock/cryptocurrency price and news sentiment analysis model.

    Input will be stock/cryptocurrency ticker along with current price, 5 days of historical daily price, news sentiment average from AlphaVantage API for latest 10 news articles, and 10 latest news articles will also be input.

    Your task is to output a summary of those 10 news articles along with estimates of closing price for next 3 days. This estimate should be based on price trends along with news sentiment analysis and knowledge gained from news articles.

    Inputs:
    ===============
    Ticker: [string]
    current_price: [float]
    historical_price_data: [list of historical closing prices for last 5 days week]
    news_sentiment_average_alphavantage: Average of news sentiment scores for latest 10 news articles from AlphaVantage API (x, where x <= -0.35: Bearish; -0.35 < x <= -0.15: Somewhat-Bearish; -0.15 < x < 0.15: Neutral; 0.15 <= x < 0.35: Somewhat_Bullish; x >= 0.35: Bullish)
    news_articles_list: list of Urls of Latest 10 news articles related to ticker from AlphaVantage API.


    Outputs (in JSON format):
    ===============
    predicted_prices: [list of predicted closing prices for next 3 days]
    summary_of_news_articles: [summary of latest 10 news articles]

    Example:
    ===============
    Input:
    ------
    ticker: "AAPL"
    current_price: 150.0
    historical_price_data: {'2024-10-28': 426,'2024-10-29': 431,'2024-10-30': 432,'2024-10-31': 406,'2024-11-01': 410}
    news_sentiment_average_alphavantage: 0.2
    news_articles_list: ["https://example.com/article1", "https://example.com/article2", ...]

    Output:
    future_prices_3_days: {'2024-11-02': 415, '2024-11-03': 419, '2024-11-04': 405}
    summary_of_news_articles: "The latest news articles related to AAPL were ..."
    """
    model = 'gemini-1.5-flash' # @param {type: "string"} ["gemini-1.5-pro", "gemini-1.5-flash"]
    temperature = 0.5 # @param {type: "slider", min: 0, max: 2, step: 0.05}

    api_key = userdata.get(api_key_name)
    genai.configure(api_key=api_key)

    model = genai.GenerativeModel(model, system_instruction=system_instructions)
    config = genai.GenerationConfig(temperature=temperature)

    prompt = f"""
    Input:
    ------
    ticker: {ticker}
    current_price: {stock_data.info['currentPrice']}
    historical_price_data: {hist_closing_prices}
    news_sentiment_average_alphavantage: 0.2
    news_articles_list: ["https://example.com/article1", "https://example.com/article2", ...]
    """

    response = model.generate_content(contents=[prompt], generation_config=config)

    print(response.text)

    # Display results in Gradio UI
    # return f"Current Price: {stock_data.info['regularMarketPrice']}\nAverage Sentiment: {avg_sentiment}\nPredicted Price: {prediction}"

# Create Gradio interface
# iface = gr.Interface(
#     fn=predict_stock_price,
#     inputs="text",
#     outputs="text"
# )

# iface.launch()

In [72]:
predict_stock_price("AAPL")

```json
{
  "predicted_prices": {
    "2024-11-02": 220,
    "2024-11-03": 223,
    "2024-11-04": 225
  },
  "summary_of_news_articles": "The latest news articles related to AAPL indicate a somewhat bullish sentiment. While the stock has experienced a slight downward trend in recent days, news articles suggest potential positive developments in the company's future. Some articles highlight the company's upcoming product launches, while others discuss strong financial performance and positive market outlook. Overall, the news suggests a potential for moderate growth in the coming days."
}
```

**Explanation:**

* **Price Trend:** The historical price data shows a downward trend over the past 5 days. However, the current price is slightly above the closing price of the previous day. This suggests a potential for a slight rebound.
* **News Sentiment:** The news sentiment score of 0.2 indicates a somewhat bullish sentiment. This suggests that the news articles are generally positive toward